In [1]:
import tensorflow as tf
import numpy as np
import time, datetime
import matplotlib.pyplot as plt
import pickle

# constants definition
CNN_NAME = 'pretrained_restnet50'
EPOCHS = 1
NUM_CLASSES = 10
BATCH_SIZE_TRAIN = 20
BATCH_SIZE_TEST = 128


# build the neural net model
def build_model():
  model = tf.keras.Sequential()
  model.add(tf.keras.layers.UpSampling2D((2,2)))
  model.add(tf.keras.layers.UpSampling2D((2,2)))
  model.add(tf.keras.layers.UpSampling2D((2,2)))
  model.add(tf.keras.applications.ResNet50(
      include_top=False,
      weights='imagenet',
      input_shape=(256, 256, 3)
  ))
  model.add(tf.keras.layers.Flatten())
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(128, activation='relu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(64, activation='relu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.BatchNormalization())
  model.add(tf.keras.layers.Dense(NUM_CLASSES, activation='softmax'))

  return model

In [ ]:
# load cifar10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

# convert to one-hot vectors
y_train = tf.keras.utils.to_categorical(y_train, NUM_CLASSES)
y_test = tf.keras.utils.to_categorical(y_test, NUM_CLASSES)

model = build_model()
model.build(input_shape=(None, 32, 32, 3))
model.summary()

# set optimizer and compile model
opt_rms = tf.keras.optimizers.RMSprop(learning_rate=2e-5)
model.compile(optimizer=opt_rms, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# training
start = time.time()

print("\tStart training [", time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()), "]\n")
train_history = model.fit(
    x_train, y_train,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE_TRAIN,
    validation_data=(x_test, y_test),
    verbose=1)
print("\n\tEnd training [", time.strftime("%Y-%m-%d %H:%M:%S", time.gmtime()), "]")

end = time.time()
print("\n\tTotal training time:", datetime.timedelta(seconds=round(end - start, 0)))

In [ ]:
# testing
scores = model.evaluate(x_test, y_test, batch_size=BATCH_SIZE_TEST, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100, scores[0]))

In [ ]:
# save to disk
with open(f"{CNN_NAME}_{EPOCHS}_model.json", 'w') as model_file:
  model_file.write(model.to_json())
model.save_weights(f"{CNN_NAME}_{EPOCHS}_weights.h5")

with open(f"{CNN_NAME}_{EPOCHS}_history.sav", 'wb') as history_file:
  pickle.dump(train_history.history, history_file)

In [ ]:
# load history
history = pickle.load(open(f"{CNN_NAME}_{EPOCHS}_history.sav", "rb"))

plt.figure(figsize=(15.0, 9.0))
plt.xlabel('Epoch')

#plt.plot(history['loss'])
#plt.plot(history['val_loss'])
#plt.title('Custom CNN loss')
#plt.ylabel('Loss')
#plt.legend(['Train', 'Validation'], loc='upper center')

plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('Custom CNN accuracy')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Validation'], loc='lower center')
plt.xticks(np.arange(len(history['accuracy'])), np.arange(1, len(history['accuracy'])+1))

plt.show()